In [1]:
import pandas as pd
import requests
import concurrent.futures
import numpy as np
import sys
from os.path import exists
import re
import itertools
import orjson
import time
from datetime import date
import urllib.parse
import os
import xml.etree.ElementTree
from bs4 import BeautifulSoup
from os.path import exists
import json
import csv
import time

In [2]:
CLEANR = re.compile('<.*?>')
def cleanhtml(raw_html):
    cleantext =re.sub(CLEANR, '', str(raw_html))
    return cleantext

In [3]:
out_csv = 'Ellementco.csv'
input_folder = "Ellementco2"
baseurl = "https://ellementco.com/"

csv_exists = exists(out_csv)
if csv_exists:
    print(f"csv exists in path {out_csv},removing now ....")
    os.remove(out_csv)
    
df = pd.DataFrame(columns=['ns_Segment','ns_Sub_Segment','ns_Sub_Sub_Segment','ns_Segment_amz','ns_price','source_offer_price','brand','series','actual_model_no','ns_model_no','ns_model_name','ns_DisplayName','ns_no_of_review','ns_review_rating','ns_model_number','source_product_id','source_affiliates','ns_B2B','breadcrumb','ns_product_name','ns_Other_forms','ns_Title','ns_upc1','ns_Ean1','ns_ean_list','manufacturer','product_warranty','Product_Description','ns_Studio','ns_Specifications','ns_Publisher','ns_Features','ns_Image_URLS','ns_URL','ns_Benefits','ns_Applications','Size','Color','Shipping','Return','ns_instock','ns_Image_Path','ns_active_flag','ns_merch_cat_path1','ns_merch_cat_path2','ns_norma_merch_cat_path1','ns_norma_merch_cat_path2','ns_salesrank','omni_b2b_psv','short_description','Description','How_to_use','Composition_and_Info_pack','SKU'])
header_list=df.columns
print(f"creating new CSV...")
df.to_csv(out_csv,sep="|",index=False,header=False)

csv exists in path Ellementco.csv,removing now ....
creating new CSV...


In [4]:
parent_dir = "/home/justdial/data/input"
path = os.path.join(parent_dir, input_folder) 
file_exists = exists(path)
if not file_exists :
        os.makedirs(path)
else:
    print("Path Already Exits")

Path Already Exits


In [222]:
def parse_prod(ppage,prod_url,main_cat,spage):
    ppage = re.sub('<!--(.*?)-->','',str(ppage))
    ppage = re.sub('&times;','x',str(ppage))
    ppage = re.sub('amp;','',str(ppage))
    ppage = re.sub('&#8211;','-',str(ppage))
    ppage = re.sub('&#8217;','',str(ppage))
    ppage = re.sub('&#038;','&',str(ppage))
    ppage = re.sub('&#8243;','Inches',str(ppage))
    #ppage = re.sub('&quot;','Inches',str(ppage))
    ppage = re.sub('&nbsp;','',str(ppage))
    try:
        prod_name = re.findall('class="product__title\s*heading-size--page-title"\s>\s*(.*?)\s*</h1>',str(ppage))[0]
        prod_name = cleanhtml(prod_name)
        print(prod_name)
    except:
        prod_name =""

    # image urls
    img_cut =  re.findall('"images":\[(.*?)\]',str(ppage),re.MULTILINE)[0]
    img_cut = re.sub('",', '',str(img_cut))
    img_cut = re.sub('\\\\','',str(img_cut))
 #   print(img_cut)
    img_urls = re.findall('"(.*?)\?',str(img_cut))
    
    newimg_urls = ' |~| '.join(img_urls)
    newimg_urls = re.sub('//','https://',str(newimg_urls))
    #print(newimg_urls)

    # color
    try:
        color = re.findall('}\'\s*data-mce-fragment="1">(.*?)</span>',str(ppage))[0]
    except:
        color = ""
    if not color:
        try:
            color = re.findall('>Color.*?<br>(.*?)</p>',str(ppage))[0]
        except:
            color = ""
    if not color:
        try:
            color = re.findall('p><b>Color:\s*(.*?)\s*</b></p>',str(ppage))[0]
        except:
            color = ""
    if not color:
        try:
            color = re.findall('</strong>:\s*(.*?)<br></p>',str(ppage))[0]
        except:
            color = ""
    if not color:
        try:
            color = re.findall('<b>olor:\s*</b>(.*?)</p>',str(ppage))[0]
        except:
            color = ""
    if not color:
        try:
            color = re.findall('Color:\s*</strong>(.*?)</p>',str(ppage))[0]
        except:
            color = ""
    #print(color)

    # how to use
    try:
        use = re.findall('class="metafield-rich_text_field"><ol><li>(.*?)</ol>')
    except:
        use = ""

    #print(use)
    
    # features
    try:
        features = re.findall('>Features\s*(.*?)\s*</div>',str(ppage))[0]
        features = re.sub('</li>\s*</ul>','',str(features))
        features = re.sub('</li>',' ~ ',str(features))
        features = cleanhtml(features)
    except:
        features = ""
    
    #print(features)

    # ingredients

    try:
        ingredients = re.findall('>Ingredients\s*(.*?)\s*</p>',str(ppage))
        ingredients = cleanhtml(ingredients)
    except:
        ingredients = ""
    print(ingredients)

In [223]:
def fetch_prod(spage,main_cat,sub_cat):
    ph = re.findall('<a class="product-item__title"\s*href="(.*?)".*?text-size--large text-weight--bold">\s*(.*?)\s*</span>',str(spage),re.MULTILINE)
    prod_hash = dict(ph)
    #print(len(prod_hash))
    for prod_url in prod_hash:
        prod_name = prod_hash[prod_url]
        
        #prod_url = baseurl+str(prod_url)
        if re.match('http',str(prod_url)):
            pass
                #print("Complete Url")
        else:
            prod_url = str(baseurl)+str(prod_url)
        print(prod_url)
        new_prod_url = prod_url
        new_prod_url = new_prod_url.replace('/','_').replace(':','_').replace('.','')
        file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+str(new_prod_url)+'.html')
        if file_exists:
            pass
        else:
            
            headers = {
                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/119.0',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                # 'Accept-Encoding': 'gzip, deflate, br',
                'Alt-Used': 'www.themaevastore.com',
                'Connection': 'keep-alive',
            }
            #time.sleep(5)
            prod_page = requests.get(f'{prod_url}', headers=headers)
            
            #prod_page = requests.get(f'{prod_url}', headers=headers)
            #prod_page = requests.get(prod_url)
            if prod_page.status_code == 200:
                    with open('/home/justdial/data/input/'+str(input_folder)+'/'+str(new_prod_url)+'.html','w')as f:

                        f.write(prod_page.text.encode(encoding="utf-8",errors='ignore').decode())
                        f.close()
        HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+str(new_prod_url)+'.html', "r")
        ppage = HTMLFile.read()
        ppage = re.sub('\\n',' ',str(ppage))
        parse_prod(ppage,prod_url,main_cat,spage)
    

In [224]:
file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html')
if file_exists:
        pass
else:
    import requests
    homepage = requests.get(baseurl)
    print(homepage.status_code)
    if homepage.status_code == 200:
            with open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html','w')as f:
                
                f.write(homepage.text.encode(encoding="utf-8",errors='ignore').decode())
                
HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html', "r")
hpage = HTMLFile.read()
hpage = re.sub('\\n','',str(hpage))
hpage = re.sub('<!--(.*?)-->','',str(hpage))
hpage = re.sub('#038;','',str(hpage))
mhash = re.findall('<a class="menu-link  " href="(.*?)".*?text-animation--underline-in-header">(.*?)<',str(hpage))
main_cat_hash = dict(mhash)
print(main_cat_hash)
for main_cat_url in main_cat_hash:
    main_cat = main_cat_hash[main_cat_url]
    sub_cat_cut = re.findall('id="SiteNavLabel-nails-classic">(.*?)</ul>',str(hpage))[0]
    shash = re.findall('href="(.*?)".*?<span class="text-animation--underline-in-header">\s*(.*?)\s*</span>',str(sub_cat_cut))
    sub_cat_hash = dict(shash)
    
    for sub_cat_url in sub_cat_hash:
        sub_cat = sub_cat_hash[sub_cat_url]
        if re.match('http',str(sub_cat_url)):
            pass
        else:
            sub_cat_url = str(baseurl)+str(sub_cat_url)
        print(sub_cat_url)
        new_sub_cat_url = sub_cat_url
        new_sub_cat_url = new_sub_cat_url.replace('/','_').replace(':','_').replace('.','')
        #print(sub_cat_url)
        file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+str(new_sub_cat_url)+'.html')
        if file_exists:
            pass
        else:
            sub_cat_page = requests.get(sub_cat_url)
            #print(sub_cat_page.status_code)
            if sub_cat_page.status_code == 200:
                    with open('/home/justdial/data/input/'+str(input_folder)+'/'+str(new_sub_cat_url)+'.html','w')as f:

                        f.write(sub_cat_page.text.encode(encoding="utf-8",errors='ignore').decode())
        HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+str(new_sub_cat_url)+'.html', "r")
        spage = HTMLFile.read()
        spage = re.sub('\\n','',str(spage))
        fetch_prod(spage,main_cat,sub_cat)

{'#': 'Nails'}
https://ellementco.com//collections/vegan-nail-polishes
https://ellementco.com//collections/vegan-nail-polishes/products/milky-way-nail-polish-10ml
Milky Way Nail Polish
['                 Nitrocellulose, Polyester Resin Acetyl tri butyl citrate, Ethyl Acetate, Butyl Acetate, Iso-Propyl Alcohol, n-Butyl Alcohol, Phenaxy Methanol Acetate, Acrylate Co Polymer, Steary Iconium, Hectorite.']
https://ellementco.com//collections/vegan-nail-polishes/products/desk-to-dinner-nude-pink-nail-polish
Desk-to-Dinner Nude Pink Nail Polish
['                 Nitrocellulose, Polyester Resin Acetyl tri butyl citrate, Ethyl Acetate, Butyl Acetate, Iso-Propyl Alcohol, n-Butyl Alcohol, Phenaxy Methanol Acetate, Acrylate Co Polymer, Steary Iconium, Hectorite.']
https://ellementco.com//collections/vegan-nail-polishes/products/cosmic-chrome-nail-polish-10ml
Cosmic Chrome Nail Polish
['                 Nitrocellulose, Polyester Resin Acetyl tri butyl citrate, Ethyl Acetate, Butyl Acetate, Iso-Pro